# Location

This notebook looks up the address, longitude and latitude of the stores.

In [ ]:
import os
import pandas as pd
import googlemaps
import numpy as np
import re

In [ ]:
df_info = pd.read_excel('images/2021/tables/image_name_information.xlsx', index_col=0)

Google maps struggles with locations that include the store number for stores that are located in a mall. Therefore we remove the store number from the location.

In [ ]:
def remove_shop_nr_from_location(location_str):
    location_split_ls = location_str.split(' ')
    if re.search(r'\d', location_split_ls[-1]):
        location_split_ls = location_split_ls[:-1]
    
    return ' '.join(location_split_ls)

In [ ]:
df_info['location'] = df_info['location'].apply(lambda location_str: remove_shop_nr_from_location(location_str))

In [ ]:
retail_rename_dict = {}

with open('retailer.txt') as f:
    for line in f.readlines():
        new_name, old_name = line.strip().split(' ')
        retail_rename_dict[new_name] = old_name

In [ ]:
df_info['retailer'] = df_info['retailer'].apply(lambda retail_name: retail_rename_dict[retail_name])

In [ ]:
maps_api_key = os.environ.get('GOOGLE_MAPS_API')
map_client = googlemaps.Client(maps_api_key)

In [ ]:
def get_address_lat_lng(location_name):
    try:
        response = map_client.places(query=location_name)
        results = response.get('results')
        address = results[0]['formatted_address']
        lat, lng = results[0]['geometry']['location'].values()
        return address, lat, lng

    except Exception as e:
        print(e)
        return np.nan, np.nan, np.nan

In [ ]:
df_info['query_location'] = df_info['retailer'] + ' ' + df_info['location']

In [ ]:
df_info['address'], df_info['lat'], df_info['lng'] = zip(*df_info['query_location'].map(get_address_lat_lng))

In [ ]:
def extract_zip_code(address_str):
    try:
        the_zip = re.search('\s[0-9]{4},', address_str)[0][1:-1]
        return the_zip
    except Exception as e:
        print(e)
        return np.nan

In [ ]:
df_info['zip'] = df_info['address'].apply(lambda address_str: extract_zip_code(address_str))

In [ ]:
def get_province(zip_code):
    try:
        zip_int = int(zip_code)
        if (1 <= zip_int <= 299) or (1400 <= zip_int <= 2199):
            return 'Gauteng'
        if (300 <= zip_int <= 499) or (2500 <= zip_int <= 2899):
            return 'Northwest'
        if 500 <= zip_int <= 999:
            return 'Limpopo'
        if (1000 <= zip_int <= 1399) or (2200 <= zip_int <= 2499):
            return 'Mpumalanga'
        if 2900 <= zip_int <= 4730:
            return 'KwaZulu-Natal'
        if 4731 <= zip_int <= 6499:
            return 'Easter Cape'
        if 6500 <= zip_int <= 8099:
            return 'Western Cape'
        if 8100 <= zip_int <= 8999:
            return 'Northern Cape'
        if 9300 <= zip_int <= 9999:
            return 'Free State'
    except Exception as e:
        print(e)
        return np.nan

In [ ]:
df_info['province'] = df_info['zip'].apply(lambda zip_code: get_province(zip_code))

In [ ]:
reverse_retail_rename_dict = {}

for key, value in retail_rename_dict.items():
    reverse_retail_rename_dict[value] = key

In [ ]:
df_info['retailer'] = df_info['retailer'].apply(lambda retail_name: reverse_retail_rename_dict[retail_name])

In [ ]:
df_info.to_excel('images/2021/tables/image_name_information_with_location.xlsx')